## Algoritmo de automatización 



In [ ]:
import json
import numpy as np
import logging
import os
import math
import copy
import random
import string
from datetime import datetime, timezone
from typing import Dict, List, Any, Optional, Tuple, Set

# --- Configuration ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)



RELAY_PAIRS_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_automation.json"
OPTIMIZED_SETTINGS_OUTPUT_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/optimized_relay_values.json"


# --- Constants ---
K = 0.14
N = 0.02
CTI = 0.2
# Optimization Bounds and Parameters
MIN_TDS = 0.05 # Lower practical limit for TDS
MAX_TDS = 1.0
MIN_PICKUP = 0.05
# Stricter pickup limit relative to Ishc, inspired by snippet
MAX_PICKUP_FACTOR = 0.7 # Max pickup = Ishc * 0.7
# MAX_TIME still needed for constraint violations, but not for calculation errors
MAX_TIME = 20.0
MAX_ITERATIONS = 250 # More iterations might be needed
# Convergence Targets
TARGET_TMT = -0.005 # Target for *total* miscoordination time (can stay negative)
# ***MODIFIED: New threshold for individual Margin Time (MT)***
MIN_ALLOWED_INDIVIDUAL_MT = -0.009 # Convergence threshold for the most negative individual MT
CONVERGENCE_THRESHOLD_TMT = 0.005 # Smaller margin for TMT (still used for stagnation check)
# Objective Function Weights
W_TIME = 0.1 # Further decrease weight of total time
W_MT = 15.0 # Increase weight for miscoordination penalty
W_PICKUP_DIFF = 0.0 # Currently unused
# Adjustment Step Sizes & Factors (Inspired by Snippet)
AGGRESSIVE_MT_THRESHOLD = -CTI * 0.75 # Threshold to trigger aggressive adjustments
# Aggressive Steps
AGGRESSIVE_TDS_BACKUP_FACTOR = 1.15 # Increase backup TDS more
AGGRESSIVE_TDS_MAIN_FACTOR = 0.90  # Decrease main TDS more
AGGRESSIVE_PICKUP_BACKUP_FACTOR = 1.05 # Increase backup pickup if TDS maxed out
# Normal Steps (additive for TDS, small mult for pickup)
NORMAL_TDS_BACKUP_ADD = 0.02
NORMAL_TDS_MAIN_SUB = 0.01
NORMAL_PICKUP_BACKUP_FACTOR = 1.01 # Currently less emphasized
NORMAL_PICKUP_MAIN_FACTOR = 0.99  # Currently less emphasized

# --- Helper Functions ---
def load_json_file(file_path: str) -> Optional[Any]:
    """Loads data from a JSON file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file: # Specify encoding
            data = json.load(file)
        logger.info(f"Archivo cargado exitosamente: {file_path}")
        return data
    except FileNotFoundError:
        logger.error(f"Error Crítico: Archivo no encontrado en la ruta especificada: {file_path}")
        return None
    except json.JSONDecodeError as e:
        logger.error(f"Error Crítico: Formato JSON inválido en el archivo: {file_path}. Detalles: {e}")
        return None
    except Exception as e:
        logger.error(f"Error inesperado al cargar el archivo {file_path}: {e}")
        return None

# MODIFIED: Returns Optional[float] - None indicates calculation error
def calculate_operation_time(I_shc: float, I_pi: float, TDS: float) -> Optional[float]:
    """
    Calculates relay operation time based on IEC standard curve.
    Returns None if calculation inputs are invalid or lead to math errors.
    Returns MAX_TIME if pickup constraint is violated or M <= 1.
    """
    # Basic Input Validation / Conditions for non-calculation
    if not (isinstance(I_shc, (int, float)) and I_shc > 0) or \
       not (isinstance(I_pi, (int, float)) and I_pi >= MIN_PICKUP) or \
       not (isinstance(TDS, (int, float)) and MIN_TDS <= TDS <= MAX_TDS):
        # logger.debug(f"Calc Error: Invalid input I_shc={I_shc}, I_pi={I_pi}, TDS={TDS}")
        return None # Return None for impossible basic inputs (e.g., non-positive Ishc, invalid TDS/pickup)

    # Pickup Constraint Check (relative to Ishc)
    # This check ensures the pickup setting is sensible relative to the fault current.
    # If I_pi is too high relative to I_shc, the relay might not operate reliably or as intended.
    # MAX_PICKUP_FACTOR defines this limit (e.g., 0.7 means pickup must be <= 70% of Ishc).
    if I_pi > I_shc * MAX_PICKUP_FACTOR:
        # logger.debug(f"Calc Constraint Violation: Pickup I_pi={I_pi:.4f} > I_shc*factor={I_shc * MAX_PICKUP_FACTOR:.4f}")
        return MAX_TIME # Return MAX_TIME for pickup constraint violation

    # Calculate M = I_shc / I_pi
    # Check for division by zero (should be prevented by I_pi >= MIN_PICKUP > 0 check)
    if I_pi <= 1e-9: # Avoid division by practically zero
         # logger.debug(f"Calc Error: Pickup near zero I_pi={I_pi}")
         return None

    M = I_shc / I_pi

    # Check for M <= 1.0 (relay should not operate for this fault current if M <= 1)
    # This happens if the fault current is less than or equal to the pickup setting.
    if M <= 1.0:
        # logger.debug(f"Non-operation: M={M:.4f} <= 1.0 (I_shc={I_shc:.2f}, I_pi={I_pi:.4f})")
        return MAX_TIME # Return MAX_TIME if M indicates non-operation

    # Calculate Time using the IEC formula: T = TDS * (K / (M^N - 1))
    try:
        denominator = M**N - 1
        # Check for near-zero denominator (M is guaranteed > 1 here)
        if abs(denominator) < 1e-9:
            # This case is less likely now M > 1 is enforced, but good practice.
            # logger.debug(f"Calc Error: Denominator near zero ({denominator}) for M={M:.4f}")
            return None # Return None for potential numerical instability

        time = TDS * (K / denominator)

        # Check for non-finite or non-positive results after calculation
        if not np.isfinite(time) or time <= 0:
            # logger.debug(f"Calc Error: Result non-finite or non-positive ({time:.4f}) for M={M:.4f}, TDS={TDS:.4f}")
            return None # Return None for invalid calculation results

        # Return the calculated time, capped by the maximum allowed time (MAX_TIME)
        return min(time, MAX_TIME)

    except (OverflowError, ValueError) as e:
        # Catch potential math errors during M**N calculation
        # logger.debug(f"Calc Error: Math exception M={M:.4f}, N={N} -> {e}")
        return None # Return None on math errors
    except Exception as e:
        # Catch any other unexpected errors during calculation
        logger.error(f"Excepción inesperada en calculate_operation_time (I_shc={I_shc}, I_pi={I_pi}, TDS={TDS}): {e}")
        return None # Return None for safety

def group_data_by_scenario(relay_pairs_data: List[Dict]) -> Dict[str, Dict[str, Any]]:
    """Groups relay pair data by scenario_id and extracts initial settings."""
    scenario_map: Dict[str, Dict[str, Any]] = {}
    processed_pairs_count = 0
    skipped_pairs_count = 0
    total_entries = len(relay_pairs_data)

    logger.info(f"Procesando {total_entries} entradas de pares de relés...")

    for i, pair_entry in enumerate(relay_pairs_data):
        scenario_id = pair_entry.get("scenario_id")
        main_relay_info = pair_entry.get('main_relay')
        backup_relay_info = pair_entry.get('backup_relay')

        # Basic validation for entry structure
        if not scenario_id or not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
            # logger.warning(f"Omitiendo entrada de par {i+1}/{total_entries}: Falta scenario_id o info de relé inválida.")
            skipped_pairs_count += 1
            continue

        if scenario_id not in scenario_map:
            scenario_map[scenario_id] = {
                "pairs_info": [],
                "initial_settings": {}, # Store initial TDS/pickup here
                "relays": set() # Keep track of all unique relays in the scenario
            }

        # Extract relay names and Ishc values
        main_relay = main_relay_info.get('relay')
        backup_relay = backup_relay_info.get('relay')
        I_shc_main = main_relay_info.get('Ishc')
        I_shc_backup = backup_relay_info.get('Ishc')

        # Validate relay names and Ishc values (must be positive for calculations)
        if not (main_relay and backup_relay and
                isinstance(I_shc_main, (int, float)) and I_shc_main > 0 and
                isinstance(I_shc_backup, (int, float)) and I_shc_backup > 0):
            # logger.warning(f"Omitiendo par {main_relay}/{backup_relay} en {scenario_id} (Entrada {i+1}): "
            #               f"Falta nombre de relé o Ishc inválido/no positivo (Main: {I_shc_main}, Backup: {I_shc_backup}).")
            skipped_pairs_count += 1
            continue

        # Add unique relays to the set for this scenario
        scenario_map[scenario_id]['relays'].add(main_relay)
        scenario_map[scenario_id]['relays'].add(backup_relay)

        # Extract and store initial settings (TDS, pickup) if not already stored
        initial_settings_scenario = scenario_map[scenario_id]['initial_settings']
        for r_name, r_info in [(main_relay, main_relay_info), (backup_relay, backup_relay_info)]:
            if r_name not in initial_settings_scenario:
                # Use .get() for safety, default to None if keys don't exist
                tds = r_info.get('TDS')
                # *** IMPORTANT: Input JSON uses 'pick_up', internal code uses 'pickup' ***
                pickup = r_info.get('pick_up')

                # Validate that TDS and pickup are numbers
                if isinstance(tds, (int, float)) and isinstance(pickup, (int, float)):
                    # Store with consistent internal keys
                    initial_settings_scenario[r_name] = {
                        'TDS_initial': float(tds),
                        'pickup_initial': float(pickup) # Store the initial pickup value
                    }
                # else:
                    # logger.warning(f"({scenario_id}) Configuración inicial TDS/pickup no encontrada o inválida para el relé '{r_name}' en la entrada de par {i+1}. "
                    #              f"Se usarán valores por defecto si es necesario más adelante.")
                    # Pass - default values will be handled in run_scenario_optimization

        # Add processed pair information needed for optimization
        scenario_map[scenario_id]['pairs_info'].append({
            "main_relay": main_relay,
            "backup_relay": backup_relay,
            "I_shc_main": float(I_shc_main),
            "I_shc_backup": float(I_shc_backup)
        })
        processed_pairs_count += 1

    logger.info(f"Datos agrupados por escenario. Total de entradas: {total_entries}, Pares procesados: {processed_pairs_count}, Pares omitidos: {skipped_pairs_count}")
    # Log scenarios found
    if scenario_map:
        logger.info(f"Escenarios encontrados: {list(scenario_map.keys())}")
    else:
        logger.warning("No se encontraron escenarios válidos después del procesamiento.")

    return scenario_map


# --- Scenario-Specific Optimization Function (MODIFIED for None handling, new convergence, AND INITIAL PICKUP CHECK) ---
def run_scenario_optimization(
    scenario_id: str,
    pairs_info: List[Dict],
    initial_settings: Dict[str, Dict[str, float]], # Contains TDS_initial, pickup_initial
    relays_in_scenario: Set[str]
) -> Dict[str, Dict[str, float]]:
    """
    Performs iterative optimization for a single scenario.
    Handles calculation errors (None), excludes errors from TMT calculation,
    uses the new individual MT convergence threshold (-0.009),
    AND enforces the initial pickup < min(Ishc) constraint.
    """

    logger.info(f"--- Iniciando optimización para Escenario: {scenario_id} ---")
    if not pairs_info:
        logger.warning(f"({scenario_id}) No hay pares válidos para procesar. Omitiendo optimización.")
        return {}
    if not relays_in_scenario:
        logger.warning(f"({scenario_id}) No hay relés definidos para el escenario. Omitiendo optimización.")
        return {}

    # --- ADDED: Pre-calculate minimum Ishc for each relay in this scenario ---
    # This helps enforce the condition pickup < Ishc for initial settings.
    min_ishc_per_relay: Dict[str, float] = {}
    for pair in pairs_info:
        main_relay = pair["main_relay"]
        backup_relay = pair["backup_relay"]
        # Use .get with a default of 0 to handle potential missing keys safely
        ishc_main = pair.get("I_shc_main", 0.0)
        ishc_backup = pair.get("I_shc_backup", 0.0)

        # Store the minimum positive Ishc seen by each relay
        if ishc_main > 0:
             min_ishc_per_relay[main_relay] = min(min_ishc_per_relay.get(main_relay, float('inf')), ishc_main)
        if ishc_backup > 0:
             min_ishc_per_relay[backup_relay] = min(min_ishc_per_relay.get(backup_relay, float('inf')), ishc_backup)
    # --- END ADDED SECTION ---


    # 1. Initialize settings - Start with MIN_TDS. Use initial Pickup from input,
    #    but adjust it if it violates pickup < min(Ishc) constraint.
    relay_settings: Dict[str, Dict[str, float]] = {}
    default_pickup = MIN_PICKUP * 1.5 # A sensible default if initial pickup is missing
    PICKUP_ADJUSTMENT_FACTOR = 0.99 # Factor to set pickup just below Ishc if needed (Ishc * 0.99)

    logger.info(f"({scenario_id}) Inicializando ajustes para {len(relays_in_scenario)} relés...")
    for relay in relays_in_scenario:
        # Determine the initial pickup value to use
        initial_pickup_val = default_pickup # Assume default initially
        relay_has_initial = relay in initial_settings and 'pickup_initial' in initial_settings[relay]

        if relay_has_initial:
            # Use the value from the input file
            initial_pickup_val = initial_settings[relay]['pickup_initial']
            # Basic validation for the loaded value
            if not isinstance(initial_pickup_val, (int, float)) or initial_pickup_val <= 0:
                 logger.warning(f"({scenario_id}) Pickup inicial inválido ({initial_pickup_val}) para relé '{relay}'. Usando defecto: {default_pickup:.4f}")
                 initial_pickup_val = default_pickup
        else:
             # Log warning only if the relay was expected to have settings (i.e., it was in pairs_info)
             logger.warning(f"({scenario_id}) Configuración inicial de Pickup no encontrada para relé '{relay}'. Usando defecto: {default_pickup:.4f}")

        # Ensure the starting pickup is at least MIN_PICKUP before checking against Ishc
        current_pickup = max(MIN_PICKUP, initial_pickup_val)

        # Get the minimum Ishc this relay sees in this scenario (if any)
        min_ishc_relay = min_ishc_per_relay.get(relay) # Returns None if relay wasn't found or had no positive Ishc

        if min_ishc_relay is not None and min_ishc_relay > 0:
            # Check if pickup >= min_Ishc (violates Ishc / pickup > 1)
            if current_pickup >= min_ishc_relay:
                # Adjust pickup to be slightly less than the minimum Ishc
                adjusted_pickup = min_ishc_relay * PICKUP_ADJUSTMENT_FACTOR
                # Ensure the adjusted value doesn't go below the absolute minimum pickup
                adjusted_pickup = max(MIN_PICKUP, adjusted_pickup)

                # Log the adjustment clearly
                logger.warning(
                    f"({scenario_id}) AJUSTE INICIAL DE PICKUP para relé '{relay}': "
                    f"Pickup inicial ({current_pickup:.4f}) era >= Min Ishc ({min_ishc_relay:.4f}). "
                    f"Ajustado a -> {adjusted_pickup:.4f}."
                )
                current_pickup = adjusted_pickup # Use the adjusted value
            # else: # Pickup is already < min_Ishc, which is good.
               # logger.debug(f"({scenario_id}) Verificación Inicial OK para relé '{relay}': Pickup ({current_pickup:.4f}) < Min Ishc ({min_ishc_relay:.4f}).")

        # else: # No valid minimum Ishc found for this relay. Cannot perform the check.
            # logger.debug(f"({scenario_id}) No se encontró Ishc mínimo válido (>0) para el relé '{relay}'. No se realizó verificación inicial pickup < Ishc.")

        # Assign the final calculated initial pickup and the minimum TDS
        relay_settings[relay] = {
            "TDS": MIN_TDS,        # Start all relays at minimum TDS
            "pickup": current_pickup # Use the original or adjusted pickup
        }
        # logger.debug(f"({scenario_id}) Relé '{relay}' inicializado: TDS={relay_settings[relay]['TDS']:.4f}, Pickup={relay_settings[relay]['pickup']:.4f}")

    # --- Optimization Loop ---
    last_tmt = float('inf') # Track Total Miscoordination Time of previous iteration
    no_improvement_streak = 0 # Counter for consecutive iterations with negligible TMT improvement
    CONVERGENCE_STREAK_LIMIT = 25 # Number of iterations without improvement to trigger stop

    logger.info(f"({scenario_id}) Iniciando bucle de optimización (max {MAX_ITERATIONS} iteraciones)...")
    for iteration in range(MAX_ITERATIONS):
        total_main_time = 0.0       # Sum of valid main relay operation times
        tmt = 0.0                   # Total Miscoordination Time (sum of negative MTs)
        miscoordination_penalty = 0.0 # Sum of squared negative MTs (for objective function)
        current_pair_results = []   # Store results for each pair in this iteration
        max_neg_mt = 0.0            # Track the most negative Margin Time found in this iteration (closest to 0 is better)
        error_pairs_count = 0       # Count pairs where time calculation failed (returned None)
        violated_pairs_count = 0    # Count pairs where at least one time was MAX_TIME

        # --- Step 1: Calculate performance for all pairs with current settings ---
        for pair in pairs_info:
            main_relay = pair["main_relay"]
            backup_relay = pair["backup_relay"]
            I_shc_main = pair["I_shc_main"]
            I_shc_backup = pair["I_shc_backup"]

            # Get current settings (should always exist after initialization)
            if main_relay not in relay_settings or backup_relay not in relay_settings:
                logger.error(f"({scenario_id}) Iter {iteration+1}: ¡Error Interno! Falta configuración para {main_relay} o {backup_relay}. Omitiendo par.")
                error_pairs_count += 1
                continue # Should not happen

            tds_main = relay_settings[main_relay]["TDS"]
            pickup_main = relay_settings[main_relay]["pickup"]
            tds_backup = relay_settings[backup_relay]["TDS"]
            pickup_backup = relay_settings[backup_relay]["pickup"]

            # Pickup values are already bounded by MIN_PICKUP during initialization and adjustment.
            # The MAX_PICKUP_FACTOR constraint is checked *inside* calculate_operation_time.

            # Calculate operation times for main and backup relays
            main_time = calculate_operation_time(I_shc_main, pickup_main, tds_main)
            backup_time = calculate_operation_time(I_shc_backup, pickup_backup, tds_backup)

            delta_t: Optional[float] = None # Time difference (Backup - Main)
            mt: Optional[float] = None      # Margin Time (delta_t - CTI)

            pair_status = "OK" # Track status for logging/debugging

            # --- Process results: Check for calculation errors or constraint violations ---
            if main_time is None or backup_time is None:
                # Case 1: Calculation Error (e.g., invalid inputs, math error)
                delta_t = None
                mt = None
                error_pairs_count += 1
                pair_status = "ERROR_CALC"
                # logger.debug(f"({scenario_id}) Iter {iteration+1}: Error cálculo par {main_relay}({main_time})/{backup_relay}({backup_time})")

            elif main_time >= MAX_TIME or backup_time >= MAX_TIME:
                # Case 2: Constraint Violation or Non-operation (Pickup too high, M<=1)
                # Assign a large penalty value to MT to indicate a severe issue, but don't use None.
                # This distinguishes it from a calculation error and ensures the pair is penalized.
                delta_t = MAX_TIME # Indicate violation in delta_t as well
                mt = -MAX_TIME * 2 # Use a large negative penalty for MT
                violated_pairs_count += 1
                pair_status = "VIOLATION_MAX_TIME"
                # Treat this as a severe miscoordination for adjustment purposes.
                # logger.debug(f"({scenario_id}) Iter {iteration+1}: MAX_TIME detectado {main_relay}({main_time})/{backup_relay}({backup_time}) -> mt={mt:.1f}")

            else:
                # Case 3: Valid finite times calculated
                delta_t = backup_time - main_time
                mt = delta_t - CTI
                pair_status = "OK" # Or "MISCOORD" if mt < 0

            # Store results for this pair
            current_pair_results.append({
                "main_relay": main_relay, "backup_relay": backup_relay,
                "I_shc_main": I_shc_main, "I_shc_backup": I_shc_backup,
                "main_time": main_time, "backup_time": backup_time, # Can be None, float, or MAX_TIME
                "delta_t": delta_t,     # Can be None, float, or MAX_TIME
                "mt": mt,               # Can be None, float, or large negative penalty
                "status": pair_status
            })

            # --- Accumulate performance metrics (only use valid, finite, non-violated results) ---
            # Add main time to total only if it's a valid, finite time < MAX_TIME
            if main_time is not None and main_time < MAX_TIME:
                total_main_time += main_time

            # Add to TMT, penalty, and track max_neg_mt ONLY if mt is a valid negative float
            # Exclude None (calc errors) and the large negative penalty (MAX_TIME violations)
            if mt is not None and mt < 0 and mt > -MAX_TIME: # Check mt is negative AND not the large penalty
                tmt += mt
                miscoordination_penalty += mt**2 # Penalize quadratically
                max_neg_mt = min(max_neg_mt, mt) # Update the most negative MT found so far

        # --- Step 2: Calculate Objective Function ---
        # OF = Penalty for Miscoordination + Penalty for Total Operation Time
        of = W_MT * miscoordination_penalty + W_TIME * total_main_time

        # Log progress periodically or if miscoordination is significant
        if (iteration + 1) % 25 == 0 or iteration == 0 or max_neg_mt < MIN_ALLOWED_INDIVIDUAL_MT :
             logger.info(
                 f"({scenario_id}) Iter {iteration+1}/{MAX_ITERATIONS}: OF={of:.4f}, "
                 f"TMT={tmt:.4f}, MaxNegMT={max_neg_mt:.4f} (Target>={MIN_ALLOWED_INDIVIDUAL_MT:.4f}), "
                 f"CalcErrors={error_pairs_count}, Violations={violated_pairs_count}"
             )

        # --- Step 3: Check Convergence ---
        # Converge if total miscoordination is near zero AND the worst individual miscoordination is acceptable.
        # Using TMT >= TARGET_TMT can be tricky if TARGET_TMT is negative. Let's focus on max_neg_mt.
        if max_neg_mt >= MIN_ALLOWED_INDIVIDUAL_MT:
            logger.info(
                f"({scenario_id}) *** Convergencia Alcanzada en Iteración {iteration + 1} ***\n"
                f"    TMT Final = {tmt:.4f} (Target TMT >= {TARGET_TMT})\n"
                f"    Peor MT Individual = {max_neg_mt:.4f} (Target >= {MIN_ALLOWED_INDIVIDUAL_MT})"
            )
            break # Exit optimization loop

        # --- Step 4: Adjust Relay Settings if Not Converged ---
        adjustments_made = False
        # Create a copy of current settings to modify; apply changes at the end
        next_relay_settings = copy.deepcopy(relay_settings)

        # Iterate through pairs that need adjustment (miscoordinated or violated)
        for pair_res in current_pair_results:
            # Skip pairs with calculation errors (mt is None) or already coordinated (mt >= 0)
            if pair_res["mt"] is None or pair_res["mt"] >= 0:
                continue

            # Adjust pairs where mt < 0 (including the large negative penalty for MAX_TIME cases)
            main_relay = pair_res["main_relay"]; backup_relay = pair_res["backup_relay"]
            I_shc_main = pair_res["I_shc_main"]; I_shc_backup = pair_res["I_shc_backup"]
            mt_val = pair_res["mt"] # This is a negative float or the large negative penalty

            # Get current settings from the *next* settings dict (in case of multiple adjustments)
            tds_main_curr = next_relay_settings[main_relay]["TDS"]
            tds_backup_curr = next_relay_settings[backup_relay]["TDS"]
            pickup_main_curr = next_relay_settings[main_relay]["pickup"]
            pickup_backup_curr = next_relay_settings[backup_relay]["pickup"]

            # Initialize new values with current ones
            new_tds_backup = tds_backup_curr
            new_pickup_backup = pickup_backup_curr
            new_tds_main = tds_main_curr
            new_pickup_main = pickup_main_curr

            adjustment_type = "Normal" # Default adjustment type

            # Apply adjustments based on severity (mt_val vs AGGRESSIVE_MT_THRESHOLD)
            # The large negative penalty (-MAX_TIME*2) will always trigger aggressive adjustment.
            if mt_val < AGGRESSIVE_MT_THRESHOLD:
                adjustment_type = "Aggressive"
                # Increase Backup TDS (make it slower)
                new_tds_backup = tds_backup_curr * AGGRESSIVE_TDS_BACKUP_FACTOR
                # Decrease Main TDS (make it faster)
                new_tds_main = tds_main_curr * AGGRESSIVE_TDS_MAIN_FACTOR

                # Aggressive Pickup Adjustment (if TDS hits limits):
                # If backup TDS maxed out, try increasing backup pickup (makes it slower/less sensitive)
                if abs(min(MAX_TDS, new_tds_backup) - MAX_TDS) < 1e-6:
                    max_allowed_pickup_backup = I_shc_backup * MAX_PICKUP_FACTOR # Respect constraint
                    new_pickup_backup = pickup_backup_curr * AGGRESSIVE_PICKUP_BACKUP_FACTOR
                    new_pickup_backup = min(max_allowed_pickup_backup, max(MIN_PICKUP, new_pickup_backup)) # Apply bounds

                # Optional: Aggressively decrease main pickup if main TDS hits min? Less common.
                # if abs(max(MIN_TDS, new_tds_main) - MIN_TDS) < 1e-6:
                #     max_allowed_pickup_main = I_shc_main * MAX_PICKUP_FACTOR
                #     new_pickup_main = pickup_main_curr * 0.98 # Example factor
                #     new_pickup_main = min(max_allowed_pickup_main, max(MIN_PICKUP, new_pickup_main))

            else: # Normal adjustment (mt is negative but not severely)
                adjustment_type = "Normal"
                # Additive adjustments for TDS
                new_tds_backup = tds_backup_curr + NORMAL_TDS_BACKUP_ADD
                new_tds_main = tds_main_curr - NORMAL_TDS_MAIN_SUB

                # Subtle pickup adjustments (less critical now with checks in calc_time and init)
                # These factors have less impact compared to TDS adjustments.
                # max_allowed_pickup_backup = I_shc_backup * MAX_PICKUP_FACTOR
                # new_pickup_backup = pickup_backup_curr * NORMAL_PICKUP_BACKUP_FACTOR
                # new_pickup_backup = min(max_allowed_pickup_backup, max(MIN_PICKUP, new_pickup_backup))
                # max_allowed_pickup_main = I_shc_main * MAX_PICKUP_FACTOR
                # new_pickup_main = pickup_main_curr * NORMAL_PICKUP_MAIN_FACTOR
                # new_pickup_main = min(max_allowed_pickup_main, max(MIN_PICKUP, new_pickup_main))


            # --- Apply and Bound Adjustments ---
            # Apply bounds (MIN/MAX TDS, MIN/MAX Pickup) immediately after calculation
            # Backup Relay: Increase time (Increase TDS or Pickup)
            next_relay_settings[backup_relay]["TDS"] = min(MAX_TDS, max(MIN_TDS, new_tds_backup))
            max_pickup_b = I_shc_backup * MAX_PICKUP_FACTOR # Max pickup constraint for backup
            next_relay_settings[backup_relay]["pickup"] = min(max_pickup_b, max(MIN_PICKUP, new_pickup_backup))

            # Main Relay: Decrease time (Decrease TDS or Pickup)
            next_relay_settings[main_relay]["TDS"] = min(MAX_TDS, max(MIN_TDS, new_tds_main))
            max_pickup_m = I_shc_main * MAX_PICKUP_FACTOR # Max pickup constraint for main
            next_relay_settings[main_relay]["pickup"] = min(max_pickup_m, max(MIN_PICKUP, new_pickup_main))

            # Log the specific adjustment made (optional, can be verbose)
            # logger.debug(
            #     f"({scenario_id}) Iter {iteration+1}: Ajuste {adjustment_type} para {main_relay}/{backup_relay} (mt={mt_val:.4f})\n"
            #     f"  Backup: TDS {tds_backup_curr:.4f}->{next_relay_settings[backup_relay]['TDS']:.4f}, "
            #     f"Pickup {pickup_backup_curr:.4f}->{next_relay_settings[backup_relay]['pickup']:.4f}\n"
            #     f"  Main:   TDS {tds_main_curr:.4f}->{next_relay_settings[main_relay]['TDS']:.4f}, "
            #     f"Pickup {pickup_main_curr:.4f}->{next_relay_settings[main_relay]['pickup']:.4f}"
            # )

            adjustments_made = True
        # --- End of Adjustment Loop for Pairs ---

        if adjustments_made:
            relay_settings = next_relay_settings # Apply the adjusted settings for the next iteration
        elif iteration > 15: # Stop if no adjustments were needed after an initial settling period
             logger.info(f"({scenario_id}) No se realizaron ajustes en iteración {iteration + 1} (después de iter 15). Deteniendo por estabilidad.")
             break

        # --- Step 5: Check for Stagnation (based on TMT improvement) ---
        # Use the original CONVERGENCE_THRESHOLD_TMT for stagnation check magnitude.
        # Check if the absolute improvement in TMT is negligible.
        if abs(tmt - last_tmt) < CONVERGENCE_THRESHOLD_TMT:
            no_improvement_streak += 1
        else:
            no_improvement_streak = 0 # Reset streak if there was improvement

        last_tmt = tmt # Update last TMT for the next iteration's check

        # Stop if TMT hasn't improved significantly for several iterations
        if no_improvement_streak >= CONVERGENCE_STREAK_LIMIT:
            logger.warning(
                f"({scenario_id}) Detenido por Estancamiento: TMT ({tmt:.4f}) no mejoró significativamente "
                f"durante {no_improvement_streak} iteraciones (umbral < {CONVERGENCE_THRESHOLD_TMT})."
            )
            logger.warning(f"({scenario_id}) Estado final: MaxNegMT={max_neg_mt:.4f} (Target>={MIN_ALLOWED_INDIVIDUAL_MT:.4f})")
            break

    # --- End of Main Optimization Loop ---

    else: # This block executes if the loop finished without 'break' (MAX_ITERATIONS reached)
        logger.warning(
            f"({scenario_id}) Optimización finalizada por alcanzar MAX_ITERATIONS ({MAX_ITERATIONS}). "
            f"La convergencia ({MIN_ALLOWED_INDIVIDUAL_MT:.4f}) pudo no haberse alcanzado."
        )
        # Log the final state
        logger.warning(f"({scenario_id}) Estado final: TMT={tmt:.4f}, MaxNegMT={max_neg_mt:.4f}, Errores={error_pairs_count}, Violaciones={violated_pairs_count}")


    # --- Step 6: Format and Return Final Optimized Settings ---
    logger.info(f"({scenario_id}) Formateando resultados finales...")
    formatted_settings = {}

    # Recalculate max Ishc per relay for final bounding (optional, but good practice)
    # Ensures final pickup isn't higher than factor*Ishc for the highest current it saw.
    # This was already handled during adjustments, but this is a final check.
    relays_max_ishc = {}
    for relay in relays_in_scenario:
        max_ishc_relay = 0
        for p in pairs_info:
            if p['main_relay'] == relay: max_ishc_relay = max(max_ishc_relay, p.get('I_shc_main', 0))
            if p['backup_relay'] == relay: max_ishc_relay = max(max_ishc_relay, p.get('I_shc_backup', 0))
        # Provide a fallback Ishc if none found (e.g., based on default pickup)
        relays_max_ishc[relay] = max_ishc_relay if max_ishc_relay > 0 else (default_pickup / MIN_PICKUP) * 1.5 # Heuristic fallback


    for relay, settings in relay_settings.items():
        final_tds = settings['TDS']
        final_pickup = settings['pickup']
        max_ishc_for_final_bound = relays_max_ishc.get(relay, (default_pickup / MIN_PICKUP) * 1.5)

        # Apply final bounds: MIN/MAX TDS, MIN_PICKUP, and MAX_PICKUP_FACTOR * max_Ishc
        final_tds_bounded = min(MAX_TDS, max(MIN_TDS, final_tds))
        max_allowed_pickup_final = max_ishc_for_final_bound * MAX_PICKUP_FACTOR
        final_pickup_bounded = min(max_allowed_pickup_final, max(MIN_PICKUP, final_pickup))

        # Check if final bounding significantly changed values (optional logging)
        # if abs(final_tds_bounded - final_tds) > 1e-5 or abs(final_pickup_bounded - final_pickup) > 1e-5:
        #     logger.debug(f"({scenario_id}) Final bounding applied to '{relay}': "
        #                 f"TDS {final_tds:.5f}->{final_tds_bounded:.5f}, "
        #                 f"Pickup {final_pickup:.5f}->{final_pickup_bounded:.5f}")

        # Format to 5 decimal places
        formatted_settings[relay] = {
            "TDS": float(f"{final_tds_bounded:.5f}"),
            "pickup": float(f"{final_pickup_bounded:.5f}")
        }

    logger.info(f"--- Optimización Finalizada para Escenario: {scenario_id} ---")
    return formatted_settings


# --- Script Execution ---
if __name__ == "__main__":
    logger.info("--- Iniciando Script de Optimización de Ajustes de Relés (v5 - Pickup Init Check & MT Threshold -0.009) ---")

    # --- 1. Load Input Data ---
    logger.info(f"Cargando pares de relés desde: {RELAY_PAIRS_PATH}")
    if not os.path.exists(RELAY_PAIRS_PATH):
        logger.error(f"¡Error Crítico! No se encontró el archivo de entrada: {RELAY_PAIRS_PATH}")
        raise SystemExit("Script detenido: Archivo de pares de relés no encontrado.")

    relay_pairs_data = load_json_file(RELAY_PAIRS_PATH)
    if relay_pairs_data is None or not isinstance(relay_pairs_data, list):
        logger.error("Error Crítico: El archivo de entrada está vacío, no es una lista JSON válida o no se pudo cargar.")
        raise SystemExit("Script detenido: Error al cargar datos de entrada.")
    if not relay_pairs_data:
        logger.error("Error Crítico: El archivo de entrada JSON está vacío.")
        raise SystemExit("Script detenido: No hay datos de pares de relés para procesar.")


    # --- 2. Group Data by Scenario ---
    logger.info("Agrupando datos por escenario...")
    scenario_data_map = group_data_by_scenario(relay_pairs_data)

    if not scenario_data_map:
        logger.error("Error Crítico: No se pudieron agrupar los datos o no se encontraron escenarios/pares válidos después del procesamiento inicial.")
        raise SystemExit("Script detenido: No hay escenarios válidos para optimizar.")


    # --- 3. Run Optimization for Each Scenario ---
    all_optimized_settings: Dict[str, Dict[str, Dict[str, float]]] = {}
    successful_scenarios = 0
    failed_scenarios = 0

    logger.info(f"Iniciando optimización para {len(scenario_data_map)} escenarios encontrados...")
    for scenario_id, scenario_data in scenario_data_map.items():
        # Validate scenario data before processing
        if not scenario_data.get('relays') or not scenario_data.get('pairs_info'):
             logger.warning(f"Omitiendo escenario '{scenario_id}': No contiene relés o pares de información válidos después del agrupamiento.")
             failed_scenarios += 1
             continue

        try:
            optimized_settings_for_scenario = run_scenario_optimization(
                scenario_id,
                scenario_data['pairs_info'],
                scenario_data['initial_settings'],
                scenario_data['relays']
            )

            if optimized_settings_for_scenario:
                all_optimized_settings[scenario_id] = optimized_settings_for_scenario
                successful_scenarios += 1
                logger.info(f"Optimización para escenario '{scenario_id}' completada exitosamente.")
            else:
                logger.warning(f"La optimización no produjo resultados válidos para el escenario: {scenario_id}")
                failed_scenarios += 1

        except Exception as e:
            logger.error(f"Error Inesperado durante la optimización del escenario '{scenario_id}': {e}", exc_info=True) # Log traceback
            failed_scenarios += 1
            # Continue to the next scenario if one fails

    # --- 4. Format and Save Results ---
    logger.info("--- Resumen de Optimización ---")
    logger.info(f"Escenarios procesados exitosamente: {successful_scenarios}")
    logger.info(f"Escenarios fallidos u omitidos: {failed_scenarios}")

    output_list = []
    if all_optimized_settings:
        logger.info("Formateando resultados optimizados en la estructura de lista deseada...")
        for scenario_id, optimized_settings in all_optimized_settings.items():
            # Generate a timestamp for the results
            current_timestamp = datetime.now(timezone.utc).isoformat(timespec='microseconds').replace('+00:00', 'Z') # ISO 8601 format with Z

            list_entry = {
                # "_id": { "$oid": generated_oid }, # Uncomment if MongoDB OID structure is needed
                "scenario_id": scenario_id,
                "timestamp": current_timestamp,
                "relay_values": optimized_settings # Contains optimized {'RelayName': {'TDS': x, 'pickup': y}}
            }
            output_list.append(list_entry)
        logger.info(f"Formato de lista creado con {len(output_list)} escenarios optimizados.")

        # --- 5. Save the results to JSON file ---
        try:
            # Ensure the output directory exists
            output_dir = os.path.dirname(OPTIMIZED_SETTINGS_OUTPUT_PATH)
            if output_dir and not os.path.exists(output_dir):
                logger.info(f"Creando directorio de salida: {output_dir}")
                os.makedirs(output_dir, exist_ok=True)

            # Write the list of results to the output JSON file
            # Use allow_nan=False to ensure standard JSON compatibility (NaN/Infinity are not valid)
            # Python's None will be converted to JSON null, which is standard.
            with open(OPTIMIZED_SETTINGS_OUTPUT_PATH, 'w', encoding='utf-8') as file:
                json.dump(output_list, file, indent=2, allow_nan=False)
            logger.info(f"Archivo con ajustes optimizados guardado exitosamente en: {OPTIMIZED_SETTINGS_OUTPUT_PATH}")

        except IOError as e:
             logger.error(f"Error de E/S al intentar guardar el archivo de salida {OPTIMIZED_SETTINGS_OUTPUT_PATH}: {e}")
        except Exception as e:
            logger.error(f"Error inesperado al guardar el archivo de salida {OPTIMIZED_SETTINGS_OUTPUT_PATH}: {e}")

    elif successful_scenarios == 0:
         logger.error("La optimización no produjo resultados exitosos para ningún escenario. No se guardó ningún archivo de salida.")
    else:
         # This case shouldn't happen if successful_scenarios > 0, but included for completeness
         logger.warning("No se encontraron ajustes optimizados para guardar, aunque algunos escenarios se completaron (revisar logs).")


    logger.info("--- Script de Optimización Finalizado ---")